In [1]:
using Serialization
using Printf
using CSV
using DataFrames
using ControlSystemsBase
using ControlTimingSafety
using LinearAlgebra: I
using Distributions: Pareto, Normal, cdf, quantile

using MATLABControlTest

push!(LOAD_PATH, "../src")
using Experiments
using Benchmarks
using ContinuousSims: nominal_trajectory

In [2]:
function get_ref(t::Real)
	t_i = floor(Int64, t / 0.1) + 1
	@boundscheck 1 ≤ t_i ≤ 1000 || throw(ArgumentError("t=$t out of bound"))
	DATA[t_i, :r]
end

function get_period(q::Real)
    e_i = ceil(Int64, q * length(E_VALUES))
	@boundscheck 1 ≤ e_i ≤ length(E_VALUES) || throw(ArgumentError("t=$t out of 		bound"))
	E_VALUES[e_i]
end

get_period (generic function with 1 method)

In [3]:
BATCHSIZE = 1000
SYS = ss(tf([3, 1],[1, 0.6, 1]))
DATA = CSV.read("../MATLABControlTest.jl/output-jumping1000-1e3-O1.csv", DataFrame)
E_VALUES = sort(DATA[:, :t]) / 100_000 * 0.2

H = 1000 * 0.1
q = 1.0
period = get_period(q)
H_steps = floor(Int64, H / period)+1
ref_values = map(get_ref, 0:period:H)
sysd = c2d(SYS, period)
x0 = zeros(sysd.nx)
print(x0)
sp = SamplerPWCET(q, H_steps)
samples = Vector{Tuple{BitVector,Float64}}(undef, BATCHSIZE)
for i in 1:BATCHSIZE
    σ = rand(sp)
    d = maximum(abs.(run_simulation(sysd, x0, ref_values; input = σ) - ref_values))
    samples[i] = (σ, d)
end
t = sort!(samples, by=x -> x[2])
t

[0.0, 0.0]

ErrorException: could not load library "\My Project\pwcet-safety\MATLABControlTest.jl\src\mpc_rtwdemo_single_mod.so"
%1 is not a valid Win32 application. 